# Imports

In [1]:
# Import librerías de manipulación y análisis de datos
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

# Import librerías de preprocesamiento
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA

# Import librerías de selección de características
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2

# Import librerías de modelos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
import lightgbm as lgb

# Import librerías de métricas
from sklearn.metrics import balanced_accuracy_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

# Import librerías para trabajar con Google Colab
from google.colab import drive

# Import librerías para manejar advertencias
import warnings

# Save Models Using Pickle
from sklearn import model_selection
import pickle


/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


# Carga de datos

In [2]:
# mount drive Gastón
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

# Abrir el path concreto
#Insert here your own path. 
!ls
%cd "/content/drive/MyDrive/6-2-C/AA/P2/Data"
!ls

import pickle

with open('/content/drive/MyDrive/6-2-C/AA/P2/Data/attrition_available_27.pkl', 'rb') as file:
    datos = pickle.load(file)


Mounted at /content/drive/
drive  sample_data
/content/drive/MyDrive/6-2-C/AA/P2/Data
attrition_available_27.pkl  final_model.sav


In [ ]:
# Conectar con Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

#Insertar tras %cd el path concreto
!ls
%cd "/content/drive/"
!ls

#Añadir el path concreto al archivo que contiene los datos
with open('/content/drive/', 'rb') as file:
    datos = pickle.load(file)

# Preprocesamiento de datos

In [3]:
# Separar las variables predictoras y la variable objetivo
X = datos.drop('Attrition', axis=1)
y = datos['Attrition']

# Codificar las etiquetas 'No' y 'Yes' en 0 y 1
le = LabelEncoder()
y = le.fit_transform(y)

# Separar los datos en conjuntos de entrenamiento y prueba (estratificado)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Separamos las variables numéricas de las categóricas
numeric_features = [
    'hrs', 'absences', 'JobInvolvement', 'PerformanceRating',
    'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'Age',
    'DistanceFromHome', 'Education', 'EmployeeID', 'JobLevel',
    'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
    'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
    'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager'
]

categorical_features = [
    'BusinessTravel', 'Department', 'EducationField', 'Gender',
    'JobRole', 'MaritalStatus'
]

#pipeline para atributos numéricos
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', RobustScaler())])


#pipeline para atributos categóricos
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ('pca', PCA(n_components=5))])

#Establecemos el modo en el que preprocesar las variables categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        ])

# Guardado de Modelo final

In [4]:
#ejecución del mejor modelo
lightgbm_f_classif_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('f_selector', SelectKBest(score_func=f_classif, k='all')),
    ('classifier', lgb.LGBMClassifier(objective='binary', random_state=77, class_weight='balanced'))
])

lightgbm_f_classif_clf.fit(X_train, y_train)

# save the model to disk
filename = 'final_model.sav'
pickle.dump(lightgbm_f_classif_clf, open(filename, 'wb'))
 
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(X_test)
print(result)


[0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0
 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1
 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1 0 0 1 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 1 0 0 1 0 0 0 0 1 0 0 